<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/Egitim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# --- 1. ADIM: ŞARTNAMEYE UYGUN "SENTETİK" VERİ ÜRETİMİ ---
# Şartname Bölüm 3.2'deki "Zenginleştirilmiş Varyant Profili" simülasyonu.

n_samples = 20000
print(f"TEKNOFEST 2026 Üniversite Şartnamesine uygun {n_samples} veri üretiliyor...")

def generate_bio_data(n):
    sequences = []
    bio_features = [] # Hidrofobiklik, Korunmuşluk, MAF, Risk Skoru
    labels = []

    for _ in range(n):
        # 1. Yerel Sekans Bilgisi (Öncesi 5 + Varyant + Sonrası 5 = 11 harf) [cite: 112]
        seq = ''.join(np.random.choice(list('ACGT'), size=11))

        # 2. Biyolojik Sayısal Veriler (Rastgele üretip mantıksal ilişki kuracağız)

        # In Silico Risk Skoru (0.0 - 1.0 arası): Yüksekse hastalık riski artar
        risk_score = np.random.beta(2, 2)

        # Popülasyon Sıklığı (MAF): Düşükse (Nadir) hastalık riski artar
        maf = np.random.exponential(0.1) # Genelde düşüktür
        if maf > 0.5: maf = 0.5

        # Evrimsel Korunmuşluk (0-10): Yüksekse o bölge önemlidir, bozulursa hastalık yapar [cite: 115]
        conservation = np.random.uniform(0, 10)

        # Hidrofobiklik Değişimi (-5 ile +5 arası)
        hydrophobicity = np.random.uniform(-5, 5)

        # --- ETIKETLEME MANTIĞI (GROUND TRUTH) ---
        # Gerçek hayatta bu kadar basit değildir ama modelin öğrenmesi için kural koyuyoruz:
        # Eğer Risk Skoru yüksek VE Korunmuşluk yüksek VE Nadir görülüyorsa -> PATOJENİK (1)
        # Aksi halde -> BENIGN (0)

        score_calc = (risk_score * 0.5) + (conservation/10 * 0.3) + ((0.5 - maf)*2 * 0.2)

        # Biraz gürültü ekleyelim ki ezberlemesin
        score_calc += np.random.normal(0, 0.05)

        if score_calc > 0.60:
            labels.append(1) # Patojenik
        else:
            labels.append(0) # Benign

        sequences.append(seq)
        bio_features.append([risk_score, maf, conservation, hydrophobicity])

    return sequences, np.array(bio_features), np.array(labels)

sequences, X_numerical, y = generate_bio_data(n_samples)

# Veri Kontrolü
print(f"Patojenik (1) Sayısı: {sum(y)}")
print(f"Benign (0) Sayısı: {len(y) - sum(y)}")

# --- 2. ADIM: PREPROCESSING (VERİ ÖN İŞLEME) ---

# A) Dizi Verisi (Sequence) İşleme
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(sequences)
X_seq = tokenizer.texts_to_sequences(sequences)
X_seq_pad = pad_sequences(X_seq, maxlen=11, padding='post') # 11 nükleotidlik pencere

# B) Sayısal Veri (Numerical) İşleme (Normalizasyon önemlidir)
# Basitçe 0-1 arasına sıkıştıralım (Min-Max Scaling benzeri)
X_num = X_numerical
# Risk skoru zaten 0-1, diğerlerini ölçeklemiyoruz şimdilik.

# Eğitim/Test Ayrımı
# Hem Sequence verisini hem Numerical veriyi ayırmalıyız
X_seq_train, X_seq_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_seq_pad, X_num, y, test_size=0.2, random_state=42
)

# --- 3. ADIM: HİBRİT MODEL MİMARİSİ (Sequence + Numerical) ---
# Şartnamede "Genetik sekans" VE "Biyolojik özellikler" istendiği için
# İki kollu (Two-Branch) bir model kuruyoruz.

# KOL 1: DNA Dizisini İşleyen 1D-CNN (Sequence Branch)
input_seq = Input(shape=(11,), name="Sequence_Input")
x1 = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=11)(input_seq)
x1 = Conv1D(filters=16, kernel_size=3, activation='relu')(x1)
x1 = GlobalMaxPooling1D()(x1)

# KOL 2: Sayısal Verileri İşleyen YSA (Numerical Branch) [cite: 113, 116, 117]
input_num = Input(shape=(4,), name="Bio_Features_Input")
# 4 özellik: Risk, MAF, Conservation, Hydrophobicity
x2 = Dense(16, activation='relu')(input_num)
x2 = BatchNormalization()(x2)

# BİRLEŞTİRME (Concatenation)
combined = Concatenate()([x1, x2])

# ORTAK KATMANLAR
z = Dense(32, activation='relu')(combined)
z = Dropout(0.3)(z) # Ezberlemeyi önlemek için
output = Dense(1, activation='sigmoid')(z)

# Modeli Oluştur
model = Model(inputs=[input_seq, input_num], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Özeti
print("\n--- TEKNOFEST HİBRİT MODEL MİMARİSİ ---")
model.summary()

# --- 4. ADIM: EĞİTİM ---
print("\nModel Eğitiliyor...")
history = model.fit(
    [X_seq_train, X_num_train], y_train, # İki giriş veriyoruz
    epochs=10,
    batch_size=64,
    validation_data=([X_seq_test, X_num_test], y_test),
    verbose=1
)

# --- 5. ADIM: TEST VE F1 SKORU  ---
print("\n--- TEST SONUÇLARI ---")
y_pred_prob = model.predict([X_seq_test, X_num_test])
y_pred = (y_pred_prob > 0.5).astype(int)

f1 = f1_score(y_test, y_pred)
print(f"TEKNOFEST Kriteri F1 Skoru: {f1:.4f}")
print("\nDetaylı Rapor:")
print(classification_report(y_test, y_pred, target_names=['Benign', 'Patojenik']))

# --- 6. ADIM: SENARYO BAZLI TEST (Özel Test Cases) ---
# Modelin mantığı çözüp çözmediğini test edelim.

print("\n--- SENARYO ANALİZİ (YENİ VAKA TESTLERİ) ---")

# Vaka 1: Yüksek Risk, Yüksek Korunmuşluk, Düşük MAF -> Patojenik Olmalı
vaka1_seq = tokenizer.texts_to_sequences(["ACGTACGTACG"]) # Rastgele seq
vaka1_seq = pad_sequences(vaka1_seq, maxlen=11)
vaka1_num = np.array([[0.95, 0.001, 9.5, 2.0]]) # Risk:0.95, MAF:0.001, Cons:9.5

# Vaka 2: Düşük Risk, Yüksek MAF (Sık görülen), Düşük Korunmuşluk -> Benign Olmalı
vaka2_seq = tokenizer.texts_to_sequences(["TTTTTTTTTTT"])
vaka2_seq = pad_sequences(vaka2_seq, maxlen=11)
vaka2_num = np.array([[0.10, 0.45, 1.0, -1.0]]) # Risk:0.10, MAF:0.45, Cons:1.0

prediction1 = model.predict([vaka1_seq, vaka1_num])[0][0]
prediction2 = model.predict([vaka2_seq, vaka2_num])[0][0]

print(f"VAKA 1 (Yüksek Risk Profili): %{prediction1*100:.2f} Patojenik -> {'PATOJENİK ⚠️' if prediction1 > 0.5 else 'HATA'}")
print(f"VAKA 2 (Temiz Profil)       : %{prediction2*100:.2f} Patojenik -> {'BENIGN ✅' if prediction2 < 0.5 else 'HATA'}")

TEKNOFEST 2026 Üniversite Şartnamesine uygun 20000 veri üretiliyor...
Patojenik (1) Sayısı: 8074
Benign (0) Sayısı: 11926

--- TEKNOFEST HİBRİT MODEL MİMARİSİ ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Sequence_Input      │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 11, 8)     │         40 │ Sequence_Input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Bio_Features_Input  │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 9, 16)     │        400 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │         80 │ Bio_Features_Inp… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 16)        │          0 │ conv1d_4[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16)        │         64 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      1,056 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         33 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,673 (6.54 KB)

 Trainable params: 1,641 (6.41 KB)

 Non-trainable params: 32 (128.00 B)


Model Eğitiliyor...
Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6385 - loss: 0.6411 - val_accuracy: 0.6205 - val_loss: 0.6359
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8793 - loss: 0.2886 - val_accuracy: 0.8790 - val_loss: 0.2751
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8886 - loss: 0.2525 - val_accuracy: 0.8910 - val_loss: 0.2500
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8887 - loss: 0.2560 - val_accuracy: 0.9018 - val_loss: 0.2269
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8921 - loss: 0.2430 - val_accuracy: 0.8985 - val_loss: 0.2296
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8937 - loss: 0.2445 - val_accuracy: 0.8932 - val_loss: 0.2435
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8924 - loss: 0.2408 - val_accuracy: 0.9010 - val_loss: 0.2228
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8897 - loss: 0.24

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# --- 1. ADIM: VERİ ÜRETİMİ ---
n_samples = 20000
print(f"TEKNOFEST 2026 Üniversite Şartnamesine uygun {n_samples} veri üretiliyor...")

def generate_bio_data(n):
    sequences = []
    bio_features = []
    labels = []

    for _ in range(n):
        # 1. Sekans (11 harf)
        seq = ''.join(np.random.choice(list('ACGT'), size=11))

        # 2. Biyolojik Sayısal Veriler
        risk_score = np.random.beta(2, 2)
        maf = np.random.exponential(0.1)
        if maf > 0.5: maf = 0.5
        conservation = np.random.uniform(0, 10)
        hydrophobicity = np.random.uniform(-5, 5)

        # Etiketleme Mantığı
        score_calc = (risk_score * 0.5) + (conservation/10 * 0.3) + ((0.5 - maf)*2 * 0.2)
        score_calc += np.random.normal(0, 0.05)

        if score_calc > 0.60:
            labels.append(1) # Patojenik
        else:
            labels.append(0) # Benign

        sequences.append(seq)
        bio_features.append([risk_score, maf, conservation, hydrophobicity])

    return sequences, np.array(bio_features), np.array(labels)

sequences, X_numerical, y = generate_bio_data(n_samples)

# --- 2. ADIM: EĞİTİM VERİSİNİ TEXT OLARAK KAYDETME ---
print(">> Veriler 'egitim_veri_seti.txt' dosyasına kaydediliyor...")

# Verileri bir DataFrame'de toplayıp dosyaya yazıyoruz
df_ham_veri = pd.DataFrame(X_numerical, columns=['Risk_Skoru', 'MAF_Sikligi', 'Korunmusluk', 'Hidrofobiklik'])
df_ham_veri.insert(0, 'DNA_Dizisi', sequences) # En başa diziyi ekle
df_ham_veri['HEDEF_ETIKET'] = y # 1: Patojenik, 0: Benign

# Dosyaya yazma (Tab ile ayrılmış TXT)
df_ham_veri.to_csv('egitim_veri_seti.txt', sep='\t', index=False)
print(">> Kayıt Başarılı! (Dosya adı: egitim_veri_seti.txt)\n")

# --- 3. ADIM: ÖN İŞLEME VE MODEL HAZIRLIĞI ---
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(sequences)
X_seq = tokenizer.texts_to_sequences(sequences)
X_seq_pad = pad_sequences(X_seq, maxlen=11, padding='post')

X_num = X_numerical # Zaten sayısal

# Eğitim ve Test Ayrımı
X_seq_train, X_seq_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_seq_pad, X_num, y, test_size=0.2, random_state=42
)

# Model Mimarisi (Hybrid)
input_seq = Input(shape=(11,), name="Sequence_Input")
x1 = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=11)(input_seq)
x1 = Conv1D(filters=16, kernel_size=3, activation='relu')(x1)
x1 = GlobalMaxPooling1D()(x1)

input_num = Input(shape=(4,), name="Bio_Features_Input")
x2 = Dense(16, activation='relu')(input_num)
x2 = BatchNormalization()(x2)

combined = Concatenate()([x1, x2])
z = Dense(32, activation='relu')(combined)
z = Dropout(0.3)(z)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[input_seq, input_num], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 4. ADIM: EĞİTİM ---
print("Model Eğitiliyor...")
model.fit([X_seq_train, X_num_train], y_train, epochs=8, batch_size=64, validation_split=0.1, verbose=1)

# --- 5. ADIM: TEST VE SONUÇLARI KAYDETME ---
print("\n>> Test işlemi yapılıyor ve sonuçlar kaydediliyor...")

# Tahminleri al
y_pred_prob = model.predict([X_seq_test, X_num_test])
y_pred = (y_pred_prob > 0.5).astype(int)

# F1 Skorunu Ekrana Bas
f1 = f1_score(y_test, y_pred)
print(f"Test F1 Skoru: {f1:.4f}")

# --- KAYIT İŞLEMİ (TEST SONUÇLARI) ---
# Test dizilerini (sayısal formattan) tekrar okunabilir harflere çevirelim
test_seqs_text = tokenizer.sequences_to_texts(X_seq_test)
# Tokenizer araya boşluk koyar (A C G T), onları siliyoruz (ACGT) ve büyük harf yapıyoruz
test_seqs_text = [s.replace(' ', '').upper() for s in test_seqs_text]

# Sonuç Tablosu Oluşturma
df_sonuc = pd.DataFrame(X_num_test, columns=['Risk_Skoru', 'MAF_Sikligi', 'Korunmusluk', 'Hidrofobiklik'])
df_sonuc.insert(0, 'Test_Edilen_DNA', test_seqs_text)
df_sonuc['Gercek_Etiket'] = y_test
df_sonuc['Tahmin_Olasiligi'] = y_pred_prob.flatten() # Virgüllü Olasılık
df_sonuc['Tahmin_Sinifi'] = y_pred.flatten()         # 0 veya 1

# Doğru bilip bilmediğini kontrol eden sütun
df_sonuc['SONUC_DURUMU'] = np.where(df_sonuc['Gercek_Etiket'] == df_sonuc['Tahmin_Sinifi'], 'DOGRU_BILDI', 'HATA_YAPTI')

# Dosyaya kaydetme
df_sonuc.to_csv('test_sonuc_raporu.txt', sep='\t', index=False)
print(">> Test sonuçları kaydedildi! (Dosya adı: test_sonuc_raporu.txt)")
print(">> Dosyayı açıp hangi veride hata yaptığını 'SONUC_DURUMU' sütunundan inceleyebilirsiniz.")

TEKNOFEST 2026 Üniversite Şartnamesine uygun 20000 veri üretiliyor...
>> Veriler 'egitim_veri_seti.txt' dosyasına kaydediliyor...
>> Kayıt Başarılı! (Dosya adı: egitim_veri_seti.txt)

Model Eğitiliyor...
Epoch 1/8


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


225/225 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6104 - loss: 0.6381 - val_accuracy: 0.6612 - val_loss: 0.5754
Epoch 2/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8708 - loss: 0.3255 - val_accuracy: 0.7469 - val_loss: 0.5376
Epoch 3/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8975 - loss: 0.2405 - val_accuracy: 0.8731 - val_loss: 0.2777
Epoch 4/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8969 - loss: 0.2446 - val_accuracy: 0.8850 - val_loss: 0.2517
Epoch 5/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8980 - loss: 0.2378 - val_accuracy: 0.8806 - val_loss: 0.2515
Epoch 6/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9010 - loss: 0.2318 - val_accuracy: 0.8850 - val_loss: 0.2413
Epoch 7/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8997 - loss: 0.2306 - val_accuracy: 0.8763 - val_loss: 0.2769
Epoch 8/8
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8997 - loss: 0.2285 - val_accuracy: 0.8881 - val_loss: 0